In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

chat = ChatOpenAI(
    temperature=1,
    model_name="gpt-5-nano",
    streaming=True,
    tiktoken_model_name="gpt-3.5-turbo",
    )

examples = [
    {
        "movie": "The Shawshank Redmption",
        "answer": "🔨⛓️🕊️"
    },
     {
        "movie": "The Dark Knight",
        "answer": "🦇🃏💥"
    },
     {
        "movie": "Inception",
        "answer": "💤🌀🎯"
    },
]

example_template  = ChatPromptTemplate.from_messages([
    ("human", "Summarize this {movie} with three emoticons"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_template,
    examples=examples
)

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=100,
    memory_key="chat_history",
    return_messages=True
)

def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]

final_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a master at summarizing emojis"),
    example_prompt,
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "Summarize the {movie} with emoticons")
])

conversation_chain = RunnablePassthrough.assign(chat_history=load_memory) | final_prompt | chat

def invoke_chain(movie):
    result = conversation_chain.invoke({
        "movie": movie
    })
    memory.save_context({"input":movie}, {"output":result.content})
    print(result)
    return result



In [ ]:
invoke_chain("Titanic")
invoke_chain("Parasite")


In [20]:
conversation_chain.invoke({
    "movie":"What movies did I ask you to summarize?"
})

AIMessageChunk(content='Here are the movies you asked me to summarize, with their emoji summaries:\n\n- The Shawshank Redemption: 🔨⛓️🕊️\n- The Dark Knight: 🦇🃏💥\n- Inception: 💤🌀🎯\n- Titanic: 🚢💔🌊\n- Titanic (second time): 🚢💔🧊\n- Parasite: 🏠🕵️💰')